<a href="https://colab.research.google.com/github/dudushi/CLA23-24/blob/main/HM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Student          | ID | mail |
| :---------------- | :------: | ----: |
| Kevin Elezi        |  s316685   | kevin.elezi@studenti.polito.it|
| Suyash Singh          |   s307798   | suyash.singh@studenti.polito.it |


# $\text{Homework2 : PageRank}$

In [ ]:
import numpy as np
from IPython.display import display
from scipy.linalg import block_diag
from scipy.linalg import eig
from numpy.linalg import eig
import matplotlib.pyplot as plt

In [ ]:
import numpy as np



def prv(eigenvector):
    n = eigenvector.shape[0]
    tol = 1e-8

    pagerank = {}
    for i in range(n):
        if np.abs(eigenvector[i]) < tol:
            eigenvector[i] = 0
        pagerank[f'Page{i+1}'] = eigenvector[i]

    # Sort by values, in descending order
    sorted_page_rank = dict(sorted(pagerank.items(), key=lambda item: item[1], reverse=True))

    for page, rank in sorted_page_rank.items():
        print(f'{page} = {rank}')  # Removed [0] from rank

# Example usage:
eigenvector = np.array([0.17391304, 0.26086957, 0.2173913 , 0.34782609])
prv(eigenvector)



Page4 = 0.34782609
Page2 = 0.26086957
Page3 = 0.2173913
Page1 = 0.17391304


### Non-Unique Rankings

In what follows we use $V_1(A)$ to denote the eigenspace for eigenvalue $1$ of a column-stochastic matrix **$A$**.

For our rankings it is desirable that the dimension of $V1_(A)$ equal one, so that there is a unique eigenvector $x$ with $\sum_{i}{xi} = 1$ that we can use for importance scores.

In [ ]:

A = np.array([
    [0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1/2],
    [0, 0, 1, 0, 1/2],
    [0, 0, 0, 0, 0]
])

eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvalues, eigenvectors

(array([ 1., -1.,  1., -1.,  0.]),
 array([[ 0.70710678, -0.70710678,  0.        ,  0.        ,  0.        ],
        [ 0.70710678,  0.70710678,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.70710678, -0.70710678, -0.40824829],
        [ 0.        ,  0.        ,  0.70710678,  0.70710678, -0.40824829],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.81649658]]))

But we want the eigenvalues = 1 in order to find the x (and we want the sum of the score = 1)

In [ ]:
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
eigenvector_for_1

array([[0.70710678, 0.        ],
       [0.70710678, 0.        ],
       [0.        , 0.70710678],
       [0.        , 0.70710678],
       [0.        , 0.        ]])

We find here that $V_1(A)$ is two-dimensional; one possible pair of basis vectors is <br> <br>
$$x = [1/2, 1/2, 0, 0, 0]^T$$ and $$y = [0, 0, 1/2, 1/2, 0]^T$$

(recall that any non-zero multiple of an
eigenvector is again an eigenvector).

### Danglin Nodes

A web with dangling nodes produces a matrix $A$ which contains one or more columns of all zeros. In this case $A$ is _column-substochastic_, that is, the column sums of $A$ are all less than or equal to one. Such a matrix must have all eigenvalues less than or equal to 1 in magnitude, but 1 need not actually be an eigenvalue for $A$.

In [ ]:
A = np.array([
    [0,1,0],
    [1/2,0,0],
    [1/2,0,0]
])

eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvalues, eigenvectors

(array([ 0.        ,  0.70710678, -0.70710678]),
 array([[ 0.        ,  0.70710678,  0.70710678],
        [ 0.        ,  0.5       , -0.5       ],
        [ 1.        ,  0.5       , -0.5       ]]))

As you can see there are no eigenvalues = 1.

Nevertheless, the pages in a web with dangling nodes can still be ranked use a similar technique. <br>
>The corresponding substochastic matrix must have a positive eigenvalue $λ ≤ 1$ and a corresponding eigenvector $x$ with **non-negative entries** (called the _Perron eigenvector_) that can be used to rank the web pages.

## $EXERCISE$ $1$
-----


In [ ]:
A_og = np.array([
    [0,0,1,1/2],
    [1/3,0,0,0],
    [1/3,1/2,0,1/2],
    [1/3,1/2,0,0],
])

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A_og)

In [ ]:
if np.sum(np.isclose(eigenvalues,1)) == 1 :
  print('dim(V1(A)) = 1 , we can surely use this eigenvector for the ranking!')
else :
  print('dim(V1(A)) > 1 , we are not sure on which eigenvector trust!')

dim(V1(A)) = 1 , we can surely use this eigenvector for the ranking!


In [ ]:
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
eigenvector_for_1

array([[0.72101012+0.j],
       [0.24033671+0.j],
       [0.54075759+0.j],
       [0.36050506+0.j]])

Any non-zero multiple of an
eigenvector is again an eigenvector. In order to make $\sum_{i}{xi} = 1$ we scale the eigenvector :

In [ ]:
pagerank_vector = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1)))
prv(pagerank_vector)

Page1 = [0.38709677]
Page3 = [0.29032258]
Page4 = [0.19354839]
Page2 = [0.12903226]


Now the solution is still valid.

_Suppose the people who own page $3$ in the web of Figure $1$ are infuriated by the fact that its importance score, computed using formula (2.1), is lower than the score of page $1$.<br>
 In an attempt to boost page $3$’s score, they create a page $5$ that links to page $3$; page $3$ also links to page $5$.<br> Does this boost page $3$’s score above that of page $1$?_

In [ ]:
A_1 = np.array([
    [0,      0,  1/2,  1/2,  0],
    [1/3,    0,    0,    0,  0],
    [1/3,  1/2,    0,  1/2,  1],
    [1/3,  1/2,    0,    0,  0],
    [0,      0,  1/2,    0,  0]
])

eigenvalues, eigenvectors = np.linalg.eig(A_1)
eigenvalues
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]

In [ ]:
# The normalization ensures that the sum of the elements in the eigenvector is 1,
# making it a valid probability distribution.
pagerank_vector = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()
pagerank_vector

array([0.24489796, 0.08163265, 0.36734694, 0.12244898, 0.18367347])

***Explanation***

**Given:**
The owners of Page $3$ have created a new Page $5$ that links to Page $3$, and in return, Page $3$ links to Page $5$. We are to investigate whether this strategy boosts the importance score of Page $3$ above that of Page $1$.

**Method:**
1. **Link Matrix Update:**
   - The link matrix has been updated to reflect the addition of Page $5$. Each page that links out divides its vote among its outgoing links. Page $2$, being a dangling node, distributes its rank evenly across all pages.
2. **PageRank Iteration:**
   - We iteratively calculate the PageRank scores using the power method, normalizing after each iteration.
3. **Analysis:**
   - After convergence, we analyze the scores to determine the effect of the new page on the importance of Page $3$.

**Link Matrix:**

\[
A = \begin{bmatrix}
    0 & 0 & \frac{1}{2} & \frac{1}{2} & 0 \\
    \frac{1}{3} & 0 & 0 & 0 & 0 \\
    \frac{1}{3} & \frac{1}{2} & 0 & \frac{1}{2} & 1 \\
    \frac{1}{3} & \frac{1}{2} & 0 & 0 & 0 \\
    0 & 0 & \frac{1}{2} & 0 & 0
\end{bmatrix}
\]


**Results:**
- The PageRank scores after adding Page 5 show that Page 3's score has indeed increased above that of Page 1, indicating the strategy's effectiveness.

**Conclusion:**
- The addition of Page 5 and the reciprocal link to Page 3 has successfully manipulated the PageRank algorithm, enhancing the importance score of Page 3.

## $EXERCISE$ $2$
-----

Construct a web consisting of three or more subwebs and verify that $dim(V_1(A))$ equals (or exceeds) the number of the components in the web.

In [ ]:
subweb1 = np.array([ # Two pages linking to each other
    [0, 1],
    [1, 0]
    ])

subweb2 = np.array([ # Circular linking in 3 pages
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0]])

subweb3 = np.array([ # Circular linking in 4 pages
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0]])

web_matrix = block_diag(subweb1, subweb2, subweb3)
n = len([subweb1,subweb2,subweb3])

eigenvalues, eigenvectors = np.linalg.eig(web_matrix)

eigenspace_dimension = np.sum(np.isclose(eigenvalues, 1))

print(f'eigenspace_dimension = {eigenspace_dimension}, n = {n}')
print()
if eigenspace_dimension == n:
    print('The eigenspace dim(V1(web_matrix)) equals the number of the components in the web.')
elif eigenspace_dimension > n:
    print('The eigenspace dim(V1(web_matrix)) exceeds the number of the components in the web.')
else :
    print('The eigenspace dim(V1(web_matrix)) DO NOT equals (or exceeds) the number of the components in the web.')

eigenspace_dimension = 3, n = 3

The eigenspace dim(V1(web_matrix)) equals the number of the components in the web.


In [ ]:
eigenvalues, eigenvectors

(array([ 1.00000000e+00+0.j       , -1.00000000e+00+0.j       ,
        -5.00000000e-01+0.8660254j, -5.00000000e-01-0.8660254j,
         1.00000000e+00+0.j       , -1.00000000e+00+0.j       ,
         8.32667268e-17+1.j       ,  8.32667268e-17-1.j       ,
         1.00000000e+00+0.j       ]),
 array([[ 7.07106781e-01+0.00000000e+00j, -7.07106781e-01+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [ 7.07106781e-01+0.00000000e+00j,  7.07106781e-01+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [ 0.00000000

In [ ]:
web_matrix

array([[0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0]])

## $EXERCISE$ $3$
-----

Add a link from page $5$ to page $1$ in the web of Figure 2. The resulting web, considered as an undirected graph, is connected. What is the dimension of $V_1(A)$? <br> <br>
Since linking page $5$ with page $1$ means joining the two subwebs $W_1$ and $W_2$, we can already imagine that $dim(V_1(A)) = 1$. Let's proof this:

In [ ]:
A_og2 = np.array([
    [0, 1, 0, 0,   0],
    [1, 0, 0, 0,   0],
    [0, 0, 0, 1, 1/2],
    [0, 0, 1, 0, 1/2],
    [0, 0, 0, 0,   0]
])

In [ ]:
# Add a link from page 5 to page 1 in the web
A_3 = np.array([
    [0, 1, 0, 0,   1/3],
    [1, 0, 0, 0,     0],
    [0, 0, 0, 1,   1/3],
    [0, 0, 1, 0,   1/3],
    [0, 0, 0, 0,     0]
])

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A_3)
eigenvalues, eigenvectors

(array([ 1., -1.,  1., -1.,  0.]),
 array([[ 7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
          0.00000000e+00,  7.92409618e-19],
        [ 7.07106781e-01,  7.07106781e-01,  0.00000000e+00,
          0.00000000e+00, -2.88675135e-01],
        [ 0.00000000e+00,  0.00000000e+00,  7.07106781e-01,
         -7.07106781e-01, -2.88675135e-01],
        [ 0.00000000e+00,  0.00000000e+00,  7.07106781e-01,
          7.07106781e-01, -2.88675135e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  8.66025404e-01]]))

In [ ]:
eigenspace_dimension = np.sum(np.isclose(eigenvalues, 1))
print(f'The eigenspace dim(V1(A_corrected2)) = {eigenspace_dimension}')

The eigenspace dim(V1(A_corrected2)) = 2


From the text we know that $dim(V_1(A))$ is always true for the special case of a **strongly connected network** (i.e., it is possible to get from any page to any other page in a finite number of steps). <br> <br>
 In fact in this case if we connect to pages $1$ and $2$, we will not be able to get back to pages $3,4,5$. This is why if we try to create a link from page $1$ to page $4$ to create a strongly connected network, we observe that $dim(V_1(Aconnected))$ $=1$

In [ ]:
#strongly connected web version : 1 link to 4 too !
A_strongly_connected = np.array([
    [0,   1, 0, 0,   1/3],
    [1/2, 0, 0, 0,     0],
    [0,   0, 0, 1,   1/3],
    [1/2, 0, 1, 0,   1/3],
    [0,   0, 0, 0,     0]
])

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A_strongly_connected)
eigenspace_dimension = np.sum(np.isclose(eigenvalues, 1))

print(f'The eigenspace dim(V1(A_connected)) = {eigenspace_dimension}')

The eigenspace dim(V1(A_connected)) = 1


In [ ]:
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]

pagerank_vector = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()
prv(pagerank_vector)

Page4 = 0.5000000000000004
Page3 = 0.5000000000000003
Page1 = 0.0
Page2 = 0.0
Page5 = 0.0


## $EXERCISE$ $4$
-----

In the web of Figure 2.1, remove the link from page $3$ to page $1$. In the resulting web page $3$ is now a dangling node. Set up the corresponding substochastic matrix and find its **largest positive (Perron) eigenvalue**. Find a non-negative Perron eigenvector for this eigenvalue, and scale the vector so that components sum to one. Does the resulting ranking seem reasonable?

Here it is the substochastic matrix :

In [ ]:
A_4= np.array([
    [0,     0,  0,  1/2],
    [1/3,   0,  0,    0],
    [1/3, 1/2,  0,  1/2],
    [1/3, 1/2,  0,    0]
])

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A_4)
eigenspace_dimension = np.sum(np.isclose(eigenvalues, 1))

print(f'The eigenspace dim(V1(A_connected)) = {eigenspace_dimension}')

The eigenspace dim(V1(A_connected)) = 0


As you can see there are no eigenvalues $=$ $1$.
Let's find the Perron eigenvalue :

In [ ]:
perron_eigenvalue = eigenvalues[eigenvalues.argmax()].real
perron_eigenvalue

0.5613532393351084

In [ ]:
perron_eigenvector = eigenvectors[:, eigenvalues.argmax()]
pagerank_vector = np.real(perron_eigenvector / np.sum(np.real(perron_eigenvector))).flatten()
prv(pagerank_vector)

Page3 = 0.4386467606648915
Page4 = 0.23200172279809958
Page1 = 0.20664503786679198
Page2 = 0.12270647867021683


## $EXERCISE$ $5$
-----

Prove that in any web the importance score of a page with no backlinks is zero.

Proof :

Starting from the beginning, we know that the PageRanking Problem reduces to finding the eigenvector of the eigenvalue $= 1$ : <br>

$$Ax=x$$

So for $n$ pages we have : <br> <br>
$$Ax =
\begin{bmatrix}
a_{11}x{1} &  a_{12}x{2}  &  . . .  &  a_{1n}x{n} \\
a_{21}x{1} &  a_{22}x{2}  &  . . .  &  a_{2n}x{n} \\
...   & ... & ... &  ...   \\
a_{n1}x{1} &  a_{n2}x{2}  &  . . .  &  a_{nn}x{n} \
\end{bmatrix}\ = \begin{bmatrix}
x{1} \\
x{2} \\
... \\
x{n} \\
\end{bmatrix}\ = x
$$
<br>

If $A$ is a square matrix, and $x$ is a non-zero vector such that $Ax=x$, then $x$ is called an eigenvector corresponding to the eigenvalue $1$.

To find $x$, you can set up and solve the following equation:
<br>

$$(A−I)⋅x=0$$
<br>
Here, $I$ is the identity matrix of the same size as $A$.<br>
The reasoning behind this is that if $Ax=x$, then $Ax−x=0$, and factoring out $x$, we get $(A−I)⋅x=0$

So, to find $x$, we to solve the system of linear equations given by the matrix equation $(A−I)⋅x=0$

Here's a step-by-step process:

 - Subtract the identity matrix II from matrix AA.
$(A−I)(A−I)$

 - Set up the system of linear equations:
$(A−I)⋅x=0$

Let's take an example of a page $m <= n$ that has no backlinks. <br>
The resulting matrix $A$ will be : <br> <br>

$$A =
\begin{bmatrix}
a_{11} &  a_{12}  &  . . .  &  a_{1n}\\
a_{21} &  a_{22}  &  . . .  &  a_{2n}\\
...   & ... & ... &  ...   \\
a_{m1} &  a_{m2} &  . . .  &  a_{mn}\\
...   & ... & ... &  ...   \\
a_{n1}&  a_{n2} &  . . .  &  a_{nn} \
\end{bmatrix}\ = \begin{bmatrix}
a_{11}&  a_{12}  &  . . .  &  a_{1n} \\
a_{21} &  a_{22} &  . . .  &  a_{2n} \\
...   & ... & ... &  ...   \\
0&  0  &  . . .  &  0 \\
...   & ... & ... &  ...   \\
a_{n1} &  a_{n2}  &  . . .  &  a_{nn} \
\end{bmatrix}\
$$
with: <br>


$$ a_{mj} = 0 $$ $$ j = 1, ..., n$$
<br>
so :
$$ Ax =
\begin{bmatrix}
a_{11}x{1} &  a_{12}x{2}  &  . . .  &  a_{1n}x{n} \\
a_{21}x{1} &  a_{22}x{2}  &  . . .  &  a_{2n}x{n} \\
...   & ... & ... &  ...   \\
0x{1} &  0x{2}  &  . . .  &  0x{n} \\
...   & ... & ... &  ...   \\
a_{n1}x{1} &  a_{n2}x{2}  &  . . .  &  a_{nn}x{n} \
\end{bmatrix}\ = \begin{bmatrix}
a_{11}x{1} &  a_{12}x{2}  &  . . .  &  a_{1n}x{n} \\
a_{21}x{1} &  a_{22}x{2}  &  . . .  &  a_{2n}x{n} \\
...   & ... & ... &  ...   \\
0 &  0  &  . . .  &  0 \\
...   & ... & ... &  ...   \\
a_{n1}x{1} &  a_{n2}x{2}  &  . . .  &  a_{nn}x{n} \
\end{bmatrix}\ = \begin{bmatrix}
x{1} \\
x{2} \\
... \\
x_{m} \\
... \\
x{n} \\
\end{bmatrix}\ = x
$$ <br> <br>
Consequently, given a page without backlinks $m$ , its PageRank score will be the following linear combination :<br> <br>


$$ x_m = \sum_{i = 1, ... , n}{a_{mi}x_i}  = \sum_{i = 1, ... , n}{0x_i}  = 0 $$


## $EXERCISE$ $6$
-----

Implicit in our analysis up to this point is the assertion that the manner in which the pages of a web $W$ are indexed has no effect on the importance score assigned to any given page.<br>
 Prove this, as follows: Let $W$ contains $n$ pages, each page assigned an index 1 through $n$, and let $A$ be the resulting link matrix. Suppose we then transpose the indices of pages $i$ and $j$ (so page $i$ is now page $j$ and vice-versa). Let $\tilde{A}$ be the link matrix for the relabelled web. <br>

### $6.1$
Argue that $\tilde{A} = PAP$, where $P$ is the elementary matrix obtained by transposing rows $i$ and $j$ of the $n × n$ identity matrix. Note that the operation $A → PA$ has the effect of swapping rows $i$ and $j$ of $A$, while $A→AP$ swaps columns $i$ and$j$. Also, $P^2 =I$, the identity matrix.

In [ ]:
#Let's create a strongly connected web where we'll swipe i = 1 with j = 4
A = np.array([
    [0,     0,  0, 1/2],
    [1/2,   0,  0, 1/2],
    [1/2, 1/2,  0,   0],
    [0,   1/2,  1,   0]
])


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]

In [ ]:
eigenspace_dimension = np.sum(np.isclose(eigenvalues, 1))

print(f'The eigenspace dim(V1(A_connected)) = {eigenspace_dimension}')
if eigenspace_dimension == 1:
  print('It is a strongly connected web!')

The eigenspace dim(V1(A_connected)) = 1
It is a strongly connected web!


Let's define a function **switch_pages** that create the new labeled web :

In [ ]:
def switch_pages(A,i,j):
  identity = np.eye(A.shape[0])
  P = identity
  P[i-1],P[j-1] = P[j-1].copy(),P[i-1].copy()

  A_switched1 = P @ A
  A_switched2 = A_switched1 @ P

  return A_switched2, P

In [ ]:
A_tilde , P = switch_pages(A,1,4)

The operation $A → PA$ has the effect of swapping rows $i$ and $j$ of $A$

In [ ]:
row_switch = P @ A
row_switch

array([[0. , 0.5, 1. , 0. ],
       [0.5, 0. , 0. , 0.5],
       [0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0.5]])

The operation $A → AP$ has the effect of swapping columns $i$ and $j$ of $A$

In [ ]:
column_switch = A @ P
column_switch

array([[0.5, 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5],
       [0. , 0.5, 0. , 0.5],
       [0. , 0.5, 1. , 0. ]])

Also, $P^2 =I$ , the identity matrix.

In [ ]:
P @ P

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

### $6.2$

Suppose that $x$ is an eigenvector for $A$, so $Ax=λx$ for some $λ$.  <br> Show that $y=Px$ is an eigenvector for $\tilde{A}$  with eigenvalue $λ$.

Knowing that $P^2 = I$ , $y = Px$ and $x = Py$ we can demonstrate :

$$Ax=λx$$
$$PAx = Pλx$$
$$PAPy = λPx$$
$$\tilde{A}y=λy$$

This demonstrates that $y$ serves as an eigenvector for matrix $\tilde{A}$, sharing the identical eigenvalue as $x$. Consequently, the significance scores for pages $i$ and $j$ remain consistent in both $x$ and $y$, while the scores for the remaining pages remain unaltered. Thus, swapping the indices of any two pages does not impact the importance scores; it merely alters their respective rows.

### $6.3$

Explain why this shows that transposing the indices of any two pages leaves the importance scores unchanged, and use this result to argue that any permutation of the page indices leaves the importance scores unchanged.

Keeping the example of swapping the $i=1$ page with the $j=4$ making $A^{4x4} :⟶ \tilde{A}^{4x4}$. <br>
We have that, given:

$$y = Px$$
<br>

$$
\begin{bmatrix}
x{4} \\
x{2}\\
x{3} \\
x{1} \\
\end{bmatrix}\ = \begin{bmatrix}
0 &  0 &  0  &  1 \\
0 &  1 &  0  &  0 \\
0 &  0 &  1 &  0 \\
1 &  0 &  0  &  0 \\
\end{bmatrix}\ \begin{bmatrix}
x{1} \\
x{2}\\
x{3} \\
x{4} \\
\end{bmatrix}\
$$ <br> <br>
By the moment that the PageRank score will be given from $\tilde{A}y = y$ as demonstrate before, we have : <br> <br>

$$
\tilde{A} = \begin{bmatrix}
a_{44} & a_{42} & a_{43} & a_{41} \\
a_{24} & a_{22} & a_{23} & a_{21} \\
a_{34} & a_{32} & a_{33} & a_{31} \\
a_{14} & a_{12} & a_{13} & a_{11} \\
\end{bmatrix}\
$$ <br>

$$x_4 = a_{44}x_4 + a_{42}x_2 + a_{43}x_3 + a_{41}x_1$$
$$x_1 = a_{14}x_4 + a_{12}x_2 + a_{13}x_3 + a_{11}x_1$$
<br>
$⟶$   **Any permutation of the page indices leaves the importance scores unchanged.**





In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A)
x = eigenvectors[:, np.isclose(eigenvalues, 1)]
print('PageRank for A :')
prv( np.real(x / np.sum(np.real(x))).flatten())

PageRank for A :
Page4 = 0.34782608695652173
Page2 = 0.26086956521739124
Page3 = 0.21739130434782614
Page1 = 0.1739130434782608


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A_tilde)
y = eigenvectors[:, np.isclose(eigenvalues, 1)]
print('PageRank for A_tilde :')
prv( np.real(y / np.sum(np.real(y))).flatten())

PageRank for A_tilde :
Page1 = 0.34782608695652173
Page2 = 0.2608695652173913
Page3 = 0.217391304347826
Page4 = 0.17391304347826092


In the $\tilde{A}$ matrix Page1 $⟷$ Page4 and viceversa. So this is proven.

## $EXERCISE$ $7$
-----

Prove that if $A$ is an $n × n$ column-stochastic matrix and $0 ≤ m ≤ 1$, then $M = (1 − m)A + mS$ is also a column-stochastic matrix.

To prove that is column stochastic, we try to directly sum the columns :

$$\sum_{i}{M_{i,j}} = (1-m)\sum_{i}{A_{i,j}}+ mS_{i,j}$$

The first summation will be equal to $1$ given that $A$ is column stochastic. In the secod one, we can substitute $S$ with $1/ n$ because all entries are the same, so at the end we have :

$$\sum_{i}{M_{i,j}} = (1-m)1+ m \frac{1}{n}n = 1 - m +m = 1$$

## $EXERCISE$ $8$
-----


Show that the product of two column-stochastic matrices is also column-stochastic.

Given $C = AB$, we know that, for each $k$ column of $C$ :

$$\sum_{j=i}^{n}{c_{j,k}}  = \sum_{j=1}^{n}(\sum_{l=1}^{n}{a_{j,l}b_{l,k}})$$

$$\quad \quad \quad =\sum_{l=1}^{n}{b_{l,k}}(\sum_{j=1}^{n}{a_{j,l}} ) $$

$$=\sum_{l=1}^{n}{b_{l,k}}$$
$$=1$$


## $EXERCISE$ $9$
-----
Show that a page with no backlinks is given importance score $\frac{m}{n}$ by formula.


Trying to calculating the score of a page $i$ that have no backlink, this means that the matrix $A_{ij}$ with $i$ fixed is :
$$A_{ij}=0 \quad  \forall{j}$$

so the PageRank score for the page $i$ without backlinks is:

$$x_i = (1-m)\sum_{j}{A_{ij}x_{j}} + m\sum_{j}{S x_{j}}$$

- The first summation is $0$, given that all entries are equal $0$ (no backlinks for page $i$)
- The second summation will make $\frac{m}{n}$, and in the summation stay only the $x_{j}$ that sum to $1$.




$$x_i=m\frac{1}{n}\sum_{j}{x_j}= \frac{m}{n}$$

## $EXERCISE$ $10$
-----
Suppose that $A$ is the link matrix for a strongly connected web of n pages
(any page can be reached from any other page by following a finite number of links). Show that $dim(V_1(A))$ = 1 as follows. Let $(A^k)_{ij}$ denote the $(i, j)$-entry of $A_k$.

### $10.1$

 $→$Show that $(A^2)_{ij} > 0$ if and only if page $i$ can be reached from page $j$ in exactly **two** steps.

We know that page $i$ is connected by page $j$ indirectly. <br>
This means that to get to page $i$ from page $j$ we will have to go through a page $k$ :

$$j→k \quad k→i $$

Now, given $(A^2)_{ij} = \sum_{k}{A_{ik}A_{kj}}$ <br>
We have that :

$$(A^2)_{ij} = \sum_{k}{A_{ik}A_{kj}}$$
$$ = a_{i1}a_{1j} +\quad ... \quad+ a_{ik}a_{kj} +\quad ... \quad+ a_{in}a_{nj}$$

We can confirm that :

- $j→k \quad  $ means that $ \quad a_{kj} > 0$
- $k→i \quad  $ means that $ \quad a_{ik} > 0$

Which means:
$$(A^2)_{ij} > 0 $$






This will prove true for any two $i,j$ pages connected by two steps. The reason is given by the fact that if there is one $k$ between two pages, given the symmetry of the matrix, the value of $k$ will be the same. Only that for the arrival $i$ page, $k$ will indicate the column position, while for the departure $j$ page, it will indicate the row position.<br>
This concept fits nicely with the multiplication of the matrix $A$ with itself, which will cause the two values $a_{kj} (j→k)$ and $a_{ik}(k→i)$ to always multiply, making $(A^{2})_{ij} > 0$ always true.


### $10.2$
$→$Show more generally that $(A^p)_{ij} > 0$ if and only if page $i$ can be reached from page $j$ in **EXACTLY** $p$ steps.

Let's start taking in consideration $(A^{3})_{ij}$. We shoulde prove that if page $i$ is connected to page $j$ in **EXACTLY** 3 steps, $(A^{3})_{ij} > 0$. <br>
This means that now we have an additional page $z$ such that:

$$j→z \quad z→k \quad k→i$$

We know for sure, thanks to the demonstration given before, that:

$$(A^2)_{kj} > 0$$

And we can rewrite the steps as :
$$j→k  \quad k→i$$


$$(A^{3})_{ij} = \sum_{k}{A_{ik}} \sum_{z}{A_{kz}A_{zj}} = \sum_{k}{A_{ik}A^2_{kj}} > 0$$

More **GENERALLY** we have that for a page $i$ linked to a page $j$ in $p$ steps we will always find ourselves in the condition of 2 steps : <br>

$$(A^p)_{ij} = \sum_{k}{A_{ik}A^{p-1}_{kj}} > 0$$



### $10.3$
$→$ Argue that $(I+A+A^2 +···+A^p)_{ij} > 0$ if and only if page $i$ can be reached from page $j$ in $p$ or fewer steps

If a page $i$ can be reached in $p$ steps from a page $j$ we already know that $(A^p)_{ij} > 0$ , consequently the equation $(I+A+A^2 +···+A^p)_{ij} > 0$ will be true. <br>
This because only one addend is positive (in this case $(A^p)_{ij}$ ) , and since all others less than $0$ cannot be.<br>
Let us remember, however, that we are talking about strongly connected web, and for this reason a page $i$ can be reached by $j$ in different ways !
If this page $j$ can reach the page $i$ both in $r$ and $s$ steps with $ r < s$, we will have both :

 $$(A^r)_{ij},(A^s)_{ij} > 0$$

 and

 $$(I+A+A^2 +···+A^r+···+A^s)_{ij} > 0$$

will be true not only for the $s$-long journey but for the $r$'s one too!

### $10.4$
$→$ Explain why **$I + A + A^2 + · · · + A^{n−1}$** is a positive matrix if the web is strongly connected.

If the web is strongly connected we know that each page can be reached from any other one. <br>
So far we have shown how the element $(A^p)_{ij}> 0$ for a page $i$ reached by $j$ in $p$ steps.
For a web composed of $n$ pages, we would have that each matrix $A^0,A^1, ..., A^{n-1}$ will have the elements > 0 if connected in $( 0 , 1, ..., n-1)$ steps.

So having a strongly connected web $→$all pages connect within $n-1$ steps$→$ $I + A + A^2 + · · · + A^{n−1}$ is a positive matrix.

### $10.5$
$→$Use the last part (and Exercise 8) so show that $B = \frac{1}{n} (I+A+A^2 +···+A^{n−1})$ is **positive** and **column-stochastic** (and hence by Lemma 3.2, $dim(V_1(B)) = 1)$.

We just proved that the matrix $I + A + A^2 + · · · + A^{n−1}$ is positive matrix $→$ $B$ is positive.
<br>
For the column-stochastic proof we could act like in the Exercise 8 and analyse $B$ as the sum of each $j$ column : <br>
<br>
$$\sum_{i}{B} = \frac{1}{n}(\sum_{i}{I} + \sum_{i}{A} +  \sum_{i}{A^2}+ ... + \sum_{i}{A^{n-1}})$$<br>

Still from Exercise 8 we can say that the product of 2 column-stochastic matrices is a column-stochastic matrix :
<br><br>
$$I, A, A^2, ... ,  A^{n-1}$$ are column-stochastic $→$ <br><br>

$$\sum_{i}{B} = \frac{1}{n}(\underbrace{1 + 1 +  1+ ... + 1})$$
$$ \quad \quad \quad \quad n$$
$$= \frac{1}{n}(n) = 1$$


### $10.6$

$→$Show that if $x ∈ V_1(A)$ then $x ∈ V_1(B)$. Why does this imply that $dim(V_1(A)) = 1$?

Knowing that $$Ax=x$$

Let's compute the eigenvector of $B$ :


$$B\tilde{x} = \tilde{x}$$


but B is equal to
$$\frac{1}{n}\sum^{n-1}_{i=0}A^i\tilde{x}=\tilde{x}$$
<br>
where the term $A^i$ can be written as ($A . . . AA$) for some $i$, and for this reason we have that if $\tilde{x}=x$,

<br>
$$A . . . AA \underbrace{Ax} = A . . .\underbrace{Ax} = . . . = x$$

$$x    \quad \quad \quad \quad        x$$

this property we can use in all the power matrix in the summation, obtaining<br>
<br>
$$Bx = \frac{1}{n}\sum^{n-1}_{i=0}A^ix = \frac{1}{n}\sum^{n-1}_{i=0}x = \frac{n}{n}x = x$$<br>

We found that if $x$ is the eigenvector of $A$ corresponding to the eigenvalue 1, it is for $B$. Being $B$ positive and column stochastic the $dim(V_1(B)) = 1$ by Lemma 3.2. <br>
If $q$ was another eigenvector of $A$ should be of $B$ too, but this is a contraddiction and so the $dim(V_1(A)) = 1$.

## $EXERCISE$ $11$
-----
Consider again the web in Figure 2.1, with the addition of a page $5$ that links to page $3$, where page $3$ also links to page $5$. Calculate the new ranking by finding the eigenvector of $M$ (corresponding to $λ = 1$) that has positive components summing to one. Use $m$ = 0.15.

In [ ]:
A = np.array([
    [0,      0,  1/2,  1/2,  0],
    [1/3,    0,    0,    0,  0],
    [1/3,  1/2,    0,  1/2,  1],
    [1/3,  1/2,    0,    0,  0],
    [0,      0,  1/2,    0,  0]
])

In [ ]:
n = 5
m = 0.15
S = np.ones((n,n))/n

In [ ]:
M = (1-m) * A + m * S

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(M)
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
pagerank_vector = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()
prv(pagerank_vector)

Page3 = 0.3488940909993593
Page1 = 0.23714058008945219
Page5 = 0.17827998867472777
Page4 = 0.13849550921111625
Page2 = 0.09718983102534466


## $EXERCISE$ $12$
-----
Add a sixth page that links to every page of the web in the previous exercise, but to which no other page links. <br>
Rank the pages using $A$, then using $M$ with $m$ = 0.15, and compare the results.

In [ ]:
A = np.array([[0  ,0  ,1/2,1/2,0  ,1/5],
            [1/3,0  ,0  ,0  ,0  ,1/5],
            [1/3,1/2,0  ,1/2,1  ,1/5],
            [1/3,1/2,0  ,0  ,0  ,1/5],
            [0  ,0  ,1/2,0  ,0  ,1/5],
            [0  ,0  ,0  ,0  ,0  ,0  ]])

In [ ]:
n = 6
m = 0.15
S = np.ones((n,n))/n
M = (1-m) * A + m * S

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
pagerank_vector_A = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()

eigenvalues, eigenvectors = np.linalg.eig(M)
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
pagerank_vector_M = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()

print('PageRank for A:')
prv(pagerank_vector_A)
print('********************')
print('PageRank for M:')
prv(pagerank_vector_M)

PageRank for A:
Page3 = 0.3673469387755102
Page1 = 0.24489795918367338
Page5 = 0.18367346938775522
Page4 = 0.12244897959183661
Page2 = 0.08163265306122457
Page6 = 0.0
********************
PageRank for M:
Page3 = 0.34017173872437517
Page1 = 0.23121206558721571
Page5 = 0.17382298895785955
Page4 = 0.13503312148083846
Page2 = 0.09476008524971118
Page6 = 0.025


In the rank $A$ and $M$, without considering the new page of this exercise, have the same classification in that of the exercise before. From the first rank we can see that the new page having no vote its score is 0, instead in the rank of $M$ it take the lowest score but is ranked.

## $EXERCISE$ $13$
-----


Construct a web consisting of two or more subwebs and determine the ranking given by formula (3.1).

In [ ]:
subweb1 = np.array([ # Two pages linking to each other
    [0, 1],
    [1, 0]
    ])

subweb2 = np.array([ # Circular linking in 3 pages
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0]])

subweb3 = np.array([ # Circular linking in 4 pages
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0]])

web_matrix = block_diag(subweb1, subweb2, subweb3)

In [ ]:
web_matrix

array([[0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [ ]:
n = web_matrix.shape[0]
m = 0.15
S = np.ones((n,n))/n
M =(1-m) * web_matrix + m * S

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(M)
eigenvector_for_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]
pagerank_vector = np.real(eigenvector_for_1 / np.sum(np.real(eigenvector_for_1))).flatten()
prv(pagerank_vector)

Page2 = 0.1111111111111115
Page1 = 0.11111111111111144
Page9 = 0.11111111111111106
Page3 = 0.11111111111111101
Page4 = 0.11111111111111101
Page5 = 0.11111111111111101
Page6 = 0.11111111111111101
Page7 = 0.11111111111111101
Page8 = 0.11111111111111101


In the $EXERCISE 2$ we have seen how ranking directly with $A$ we obtained 3 eigenvectors. Thanks to the new matrix $M$ we have a unique eigenvector, that allow us to ranked the web.

## $EXERCISE$ $14$
------------

For the web in Exercise 11, compute the values of $||M^kx_0-q||_1$ and $\frac{||M^kx_0-q||_1}{||M^{k-1}x_0-q||_1}$ for $k$ = 1,5,10,50, using an initial guess $x_0$ not too close to the actual eigenvector $q$ (in order to see the convergence).
<br>
Determince $c = max_{1\leq j \leq n}|1-2min_{1\leq i \leq n}M_{ij}|$ and the absolute value of the second largest eigenvalue of $M$.

In [ ]:
A = np.array([
    [0,      0,  1/2,  1/2,  0],
    [1/3,    0,    0,    0,  0],
    [1/3,  1/2,    0,  1/2,  1],
    [1/3,  1/2,    0,    0,  0],
    [0,      0,  1/2,    0,  0]
])

n = 5
m = 0.15
S = np.ones((n,n))/n

M = (1-m) * A + m * S

eigenvalues, eigenvectors = np.linalg.eig(M)
eigenvector_1 = eigenvectors[:, np.isclose(eigenvalues, 1)]

q = np.real(eigenvector_1 / sum(eigenvector_1))

In [ ]:
x_0 = np.array([
    [1],
    [0],
    [0],
    [0],
    [0]
])

In [ ]:
error_array = np.array([])
error_ratio_array = np.array([])

# Step 1: Initialize a random vector
x_k = x_0

for k in range(50):
    # Step 2: Compute y = Ab_k
    y = M @ x_k

    # Step 3: Normalize the vector
    x_k1 = y / np.linalg.norm(y,1)

    # Step 4: Compute errors
    error = np.linalg.norm(x_k1 - q, 1)
    error_array = np.append(error_array, error)

    error_ratio = error / np.linalg.norm(x_k - q, 1)
    error_ratio_array = np.append(error_ratio_array, error_ratio)

    if np.linalg.norm(x_k1 - q) < 1e-16:
            break

    x_k = x_k1

In [ ]:
print(f'x_k: {x_k}')
print(f'q:   {q}')

x_k: [[0.23714058]
 [0.09718983]
 [0.34889409]
 [0.13849551]
 [0.17827999]]
q:   [[0.23714058]
 [0.09718983]
 [0.34889409]
 [0.13849551]
 [0.17827999]]


As we can see the last $x_k$ vector is really similar to $q$. This means that the Power Method worked correctly.

In [ ]:
ks = [1,5,10,50]
for k in ks:
  print(f'STEP {k}:')
  print('***************')
  print(f'Error : {error_array[k-1]}')
  print(f'Error_Ratio : {error_ratio_array[k-1]}')

STEP 1:
***************
Error : 0.7819626528604117
Error_Ratio : 0.5125208081930114
STEP 5:
***************
Error : 0.006871410715895657
Error_Ratio : 0.3227085473885674
STEP 10:
***************
Error : 0.0003043716553394332
Error_Ratio : 0.5245287479654518
STEP 50:
***************
Error : 7.435024818036595e-13
Error_Ratio : 0.6108824300749136


We can see from this table how the error decreases as $k$ increases. And the error ratio converges to the second eigenvalue of $M$ : <br> <br>

$$λ \approx 0.611$$


In order to find constant $c$ we use the formula <br>

$$c = max_{1\leq j \leq n}|1-2min_{1\leq i \leq n}M_{ij}|$$

In [ ]:
c = np.max(1-2*np.min(M,1))
print(c)

0.94


## $EXERCISE$ $15$
------------


## $EXERCISE$ $16$
------------


Consider the link matrix $$A = \begin{bmatrix}
0 &  1/2 &  1/2 \\
0 &  0 &  1/2   \\
1 &  1/2 & 0\\
\end{bmatrix}$$


<br>

Show that $M=(1−m)A+mS $  (all $S_{ij} =1/3$) is not diagonalizable for $0≤m<1$.

**A matrix is diagonalizable if and only if the algebraic multiplicity equals the geometric multiplicity of each eigenvalues $λ_i$.**

- Algebraic Multiplicity $α$ = number of times that each eigenvalue $λ_i$ occurs in the Jordan normal form $J(A)$
- Geometric Multiplicity $β$ =  is the dimension of the corresponding eigenspace $dim(V_{λ}(A))$. In other words, it represents the number of linearly independent eigenvectors associated with a specific eigenvalue.

Now we'll prove that $M$ is not diagonqalizable by contradiction $⇒ α \neq β$

Let's start by defining $M$ <br>
$$M = (1−m)A+mS$$ <br>

$$= \begin{bmatrix}
0 &  \frac{1-m}{2} &  \frac{1-m}{2} \\
0 &  0 &  \frac{1-m}{2}  \\
1-m &  \frac{1-m}{2}& 0\\
\end{bmatrix} + \begin{bmatrix}
\frac{m}{3} & \frac{m}{3} &  \frac{m}{3} \\
\frac{m}{3}&  \frac{m}{3} &  \frac{m}{3}   \\
\frac{m}{3}&  \frac{m}{3}& \frac{m}{3}\\
\end{bmatrix}$$ <br>

$$= \begin{bmatrix}
\frac{m}{3} & \frac{3-m}{6} &  \frac{3-m}{6} \\
\frac{m}{3}&  \frac{m}{3} &  \frac{3-m}{6}   \\
\frac{3-2m}{3}&  \frac{3-m}{6}& \frac{m}{3}\\
\end{bmatrix}$$

<br>

From this result we have that the eigenvalues and the correspondent eigenvectors are : <br>



$$ \begin{array}{c} \lambda _1=1\text{, }V_{\lambda _1}\text{=Vect}\left( \begin{array}{l} \left( \begin{array}{c} -\frac{3-m}{2 (m-2)} \\ -\frac{1}{m-2} \\ 1 \end{array} \right) \end{array} \right) \\ \lambda _2=\frac{m-1}{2}\text{, }V_{\lambda _2}\text{=Vect}\left( \begin{array}{l} \left( \begin{array}{c} 0 \\ -1 \\ 1 \end{array} \right) \end{array} \right) \end{array} $$

and also : $$J(M) = \begin{bmatrix}
1 & 0 &  0 \\
0&  \frac{m-1}{2} &  1  \\
0&  0& \frac{m-1}{2}\\
\end{bmatrix}$$



$α_1 = 1, α_2 = 2$ <br>
$β_1 = dim(V_{\lambda_1}(M)) = 1, β_2 = dim(V_{\lambda_2}(M)) = 1$


We then check whether $$α_i = β_i \quad \quad ∀   λ_i$$

for $\lambda_1 = 1 \quad  ⇒ \quad \quad α_1 = 1 = β_1$ <br>
for $\lambda_2 = \frac{m-1}{2} ⇒ \quad \quad α_2 = 2 \neq β_2 = 1$ <br>

When the geometric multiplicity of a repeated eigenvalue is strictly less than its algebraic multiplicity, then that eigenvalue is said to be defective $⇒ λ_2$ is a defective eigenvalue.

$⇒$ $M$ is not diagonalizable.

Note that these conclusions are true for any value of $0≤m<1$ !

## $EXERCISE$ $17$
------------


How should the value of $m$ be chosen? How does this choice affect the rankings
and the computation time?

### $17.1$ _Computational reasons_

Traditionally, computing the PageRank vector has been viewed as an eigenvector problem,<br>

$$Mx=x$$

and Power Method has been the
method of choice. Brin and Page, the founders of Google, use $m = 0.15$ <br>
Thus, a rough estimate of the **number of iterations** needed to converge to a tolerance level $τ$ , measured by the residual:
$$x_kM-x_k = x_{k+1} - x_{k}$$ <br>
is
$$\frac{log_{10}τ}{log_{10}(1-m)}$$ <br>
And we have these results :



$$τ = 10^{-6}, m = 0.15 \quad ⟶ \quad \frac{-6}{log_{10}(.85)} \approx 85 $$ <br>

Where this number is the number of iterations until there is a convergence to the PageRank vector. We have for $τ = 10^{-8}$, about $114$ iterations and for $τ = 10^{-10}$, about $142$ iterations.
<br> <br>
This means that Google can dictate the rate of convergence according to how big $m$ is chosen to be! <br>Consequently, Google engineers are forced to perform a delicate balancing act. The bigger $m$ is, the faster the convergence, but the bigger $m$  is, the less the true hyperlink structure of the web is used to determine web page importance (17.2). And slightly different values for $m$ can produce very different PageRanks. <br>
Moreover, as $m → 0$, not only does convergence slow drastically, but sensitivity issues begin to surface as well. <br> <br>
There are good reasons for using $m = 0.15$, one being the speedy convergence of the power method. With this value for $m$, we can expect the power method to converge to the PageRank vector in about 114 iterations for a convergence tolerance level of $τ = 10^{−8}$. Obviously, this choice of α brings faster convergence than lower values of $m$. Compare with $m = 0.01$, whereby roughly $1833$ iterations are required to achieve a residual less than $10^{−8}$.<br>

When working with a sparse 4.3 billion by 4.3 billion matrix, each iteration counts; over a few hundred power iterations is more than Google is willing to compute.  


### $17.2$ _Web surfing behavior_

However, in addition to the computational reasons for choosing $m = 0.15$, this choice for $m$ also carries some intuitive weight: $m = 0.15$ implies that roughly $\frac{5}{6}$ of the time a web surfer randomly clicks on hyperlinks (i.e., following the structure of the web, as captured by the $(1-m)A$ part), while $\frac{1}{6}$ of the time this web surfer will go to the URL line and type the address of a new page to “teleport” to (as captured by the $mS$ part of the formula). Perhaps **this was the original motivation** behind Brin and Page’s choice of $m = 0.15$; it produces an accurate model for web surfing behavior.<br>
Alternatively, $m = 0.01$ not only slows convergence of the power method, but also places much greater emphasis on the hyperlink structure of the web and much less on the teleportation tendencies of surfers.

References :
1. https://www.statlect.com/matrix-algebra/algebraic-and-geometric-multiplicity-of-eigenvalues
2. https://math.stackexchange.com/questions/132552/showing-a-matrix-is-not-diagonalizable
3. https://www.stat.uchicago.edu/~lekheng/meetings/mathofranking/ref/langville.pdf